# Leonard
Leonard is a functional learning neural network built to learn how to take the coordinate of a point in space and construct a set of rotation angles that map to a kinematic chain to produce an appropriate end effector position.
> *Leonard is named for Leonard Hofstadter from the Big Bang Theory, which is a show that I have never watched.*

## v1
v1 is the experimental phase of leonard. SHort summary of progress can be found on each file

## v1.2
* Will need to optimize for GPU in ~~v1.2~~
    * Project for a v2 build for sure
    * Prefetching implemented in v1.2. Potential for GPU speed gains
* Sequential API model
    * ~~v1.1~~ will change to subclassing Model
        * ~~v1.2~~
            * ~~Delayed again: still no good reason to do this. Maybe in v2.3~~
                * Undelayed: switched to subclassing
                    * Changes will continue to be made in code structure and potentially in network structure
    * Other suitable changes to architecture will be made
* n-straight iterations; no epochs implemented
    * > Epochs will not be implemented. 
    * No tracking metrics
        * Some progress has been made here: rolling 100-avg. loss, maxloss on batch
* Implement curriculum learning
    * v1.2 project, probably.
        * In progress
* No universal test dataset yet
    * Will be used to benchmark ~~all~~ future models
        * To be built for ~~v1.1~~
            * ~~Don't worry about curriculum learning, dataset will be updated in v1.2 when curriculum is developed~~
                * Dataset will be finished for v1.2 before moving on to v1.3 or v2.0
            
* Rudimentary [Training Loop](https://console.paperspace.com/pablo6400/notebook/rpz97cf0w8bv7ns?file=%2Fleo.v1.ipynb#Training-Loop)
* Loss function based on Euclidean Distance; ~~v1.1 will regress each axis individually~~
    * Not possible. ~~Switching to MSE~~
        * Outcomes were worse
* Fixed issue where euclidean loss function was apparently reducing over entire batch. 
    * Appears to have significantly impacted training outcomes
* Switched random sampling on datagenerator to use gaussian distribution due to rotational invariance
* Attempted switch to mean squared error
    * Outcomes were worse

## The plan for v2
v2 will be the final incarnation of Leonard. Major differences:
* Penalization for long motor movements
* Constraints on movement
     * Maybe add small noise to coax regeneration to a spot where the problem is solveable?
* Measurements fully accurate to model
* Hopefully I will make better use of markdown in the notebook

In [262]:
import tensorflow as tf
import numpy as np
from collections import deque
import time

# Parameters
## Hyperparameters
 - Learning rate
 - Batch size
 - Train dataset size
 - (currently unused) dropout rate between central layers
 
## Arm Parameters

In [263]:
hyperparameters = {
    "learning_rate": 1e-3,
    "batch_size": 32,
    "train_dataset_size": 15000,
    "corpus_callosum": 0.0,
    "replay_max_probability": 0.8,
    "replay_buffer_length": 1000,
    "jitter_multiplier": 0.01,
}

In [264]:
armparameters = {
    "seglengths": [1,1,1,1,1],
}

# $f^{-1}(x)$
> The general idea is that  
> $f^{-1}(f(x)) = x$, so  
> loss = lossfn($f^{-1}(f(x))$, $x$)  
> Where $f(x)$ is the function that the neural network is trying to learn.

I am calling this ***Inverse Functional Training***, or ***IFT***


In [265]:
@tf.function
def eudist(x,y):
    return tf.norm(y-x,axis=1)

In [266]:
@tf.function
def build_rotary_matrix(angle):
  r1 = tf.stack([tf.cos(angle), 0., tf.sin(angle)])
  r2 = tf.constant([0.,1.,0.],dtype="float32")
  r3 = tf.stack([-1*tf.sin(angle),0.,tf.cos(angle)])
  return tf.stack([r1,r2,r3])
@tf.function
def build_joint_matrix(angle):
  r1 = tf.constant([1., 0., 0.],dtype="float32")
  r2 = tf.stack([0.,tf.cos(angle),-1*tf.sin(angle)])
  r3 = tf.stack([0.,tf.sin(angle),tf.cos(angle)])
  return tf.stack([r1,r2,r3])
@tf.function
def fwd(rotation_angles): # I am coming back for you: you will be so optimized later dude
    t = tf.transpose(rotation_angles)
    rotary1_matrices = tf.vectorized_map(build_rotary_matrix, t[0],fallback_to_while_loop=False)
    joint1_matrices = tf.matmul(rotary1_matrices, tf.vectorized_map(build_joint_matrix, t[1],fallback_to_while_loop=False))
    joint2_matrices = tf.matmul(joint1_matrices, tf.vectorized_map(build_joint_matrix, t[2],fallback_to_while_loop=False))
    rotary2_matrices = tf.matmul(joint2_matrices, tf.vectorized_map(build_rotary_matrix, t[3],fallback_to_while_loop=False))
    joint3_matrices = tf.matmul(rotary2_matrices, tf.vectorized_map(build_joint_matrix, t[4],fallback_to_while_loop=False))
    
    segd = tf.constant([[0],[1],[0]],dtype="float32") # TODO change this to match arm measurements
    
    seg1 = tf.matmul(rotary1_matrices, segd)
    seg2 = tf.matmul(joint1_matrices, segd)
    seg3 = tf.matmul(joint2_matrices, segd)
    seg4 = tf.matmul(rotary2_matrices, segd)
    seg5 = tf.matmul(joint3_matrices, segd)
    
    a = tf.concat([seg1,seg2,seg3,seg4,seg5],axis=2)
    b = tf.reduce_sum(a,axis=2)
    return b

# Data Pipeline
### coord_datagen() > coord_dataset
### replay ---------^
Chance of experience replay is proportional to length of replay buffer

* Should there be a recency bias?
    * How would that be implemented?
    * Is it necessary for such a short queue
* v1.2 implements prefetching, autotuned.

In [267]:
replay = deque(maxlen=hyperparameters["replay_buffer_length"]) # stores inputs in the form [[1,2,3,4,5],[...]...]

In [268]:
batch_size = hyperparameters["batch_size"]
replay_max_prob = hyperparameters["replay_max_probability"]
replay_max_len = hyperparameters["replay_buffer_length"]
train_dataset_size = hyperparameters["train_dataset_size"]

def coord_datagen(): # outputs a batch of coordinates that are approximately uniformly distributed in end effector space.
    for i in range(batch_size * train_dataset_size):  
        replay_buff_len = len(replay)
        if (np.random.uniform() < (replay_buff_len / (replay_max_len * replay_max_prob**-1))):
            r = replay.pop()
        else:
            r = tf.random.uniform(minval=-np.pi, maxval=np.pi, shape=(5,))
        yield r
    

In [269]:
coord_dataset = tf.data.Dataset.from_generator(
    coord_datagen, 
    output_signature=tf.TensorSpec((5,),dtype="float32")
).batch(hyperparameters["batch_size"]).prefetch(tf.data.AUTOTUNE)

# Model Definition
leo is defined here.

In [270]:
class Leonard(tf.keras.Model):
    def __init__(self):
        super().__init__()
        
        self.D2 = tf.keras.layers.Dense(192)
        self.BN2 = tf.keras.layers.BatchNormalization()
        # activation
        
        self.D3 = tf.keras.layers.Dense(192)
        self.BN3 = tf.keras.layers.BatchNormalization()
        # activation
        
        self.D4 = tf.keras.layers.Dense(192)
        self.BN4 = tf.keras.layers.BatchNormalization()
        # activation
        
        self.corpus_callosum = tf.keras.layers.Dropout(hyperparameters["corpus_callosum"])
        
        self.D5 = tf.keras.layers.Dense(192)
        self.BN5 = tf.keras.layers.BatchNormalization()
        # activation
        
        self.D6 = tf.keras.layers.Dense(192)
        self.BN6 = tf.keras.layers.BatchNormalization()
        # activation
        
        self.D7 = tf.keras.layers.Dense(5,kernel_regularizer="l2",dtype="float32")
        
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=hyperparameters["learning_rate"])
        self.kicked_in = False
        # self.collecting = false
    def call(self,x,training=False):
        # x /= 5
        y = tf.keras.activations.swish(self.D2(x))
        y = tf.keras.activations.swish(self.D3(y))
        y = tf.keras.activations.swish(self.D4(y))
        y = self.corpus_callosum(y,training=training)
        y = tf.keras.activations.swish(self.D5(y))
        y = tf.keras.activations.swish(self.D6(y))
        y = self.D7(y)
        return y
    
    @tf.function
    def grads(self, g):
        self.optimizer.apply_gradients(zip(g, self.trainable_variables))
        
leo = Leonard()
leo.build(input_shape=(hyperparameters["batch_size"], 3))

In [271]:
leo.load_weights("leo_v1-2-3/step_40000.keras") # start from hard 2000

# Training Loop

In [272]:
%load_ext line_profiler
def train():
    
    # qinator = deque(maxlen=1000)
    # losslog = open("losslog.txt","a")
    
    for f,x in enumerate(coord_dataset):
        fx = fwd(x)
        with tf.GradientTape() as tape:
            y = leo(fx,training=True)
            loss = eudist(fwd(y),fx)
        gradient = tape.gradient(loss, leo.trainable_variables)
        leo.grads(gradient)

        mmax = tf.reduce_max(loss)
        mloss = tf.reduce_mean(loss)
        # qinator.append(mloss)
        # mean = tf.Variable(0., trainable=False)

        if f > 1000:
            v = tf.boolean_mask(x,(loss>1))
            if (tf.size(v) > 0):
                for ex in v:
                    replay.append(ex)

        if f%1000==0: # periodic backup save
            leo.save(("leo_v1-2-3-hard/step_"+str(f)+".keras"))
        percentage = int((f/hyperparameters["train_dataset_size"])*20)
        h = hyperparameters["train_dataset_size"]
        tf.print("\r["+"="*percentage + ">" + " "*(20-percentage) + "]","Sample: {f}/{h}, Sampl. Loss: {l:.4f}, Max Loss: {mxl:.4f}".format(f=f,l=mloss,mxl=mmax,h=hyperparameters["train_dataset_size"]),end="")
        # losslog.write(str(l2loss.numpy()) + "\n")
    # losslog.close()
    
%lprun -f train train()

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
[===>                 ] Sample: 2718/15000, Sampl. Loss: 0.0364, Max Loss: 0.1177*** KeyboardInterrupt exception caught in code being profiled.

Timer unit: 1e-09 s

Total time: 190.552 s
File: /tmp/ipykernel_40/1958280877.py
Function: train at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           def train():
     3                                               
     4                                               # qinator = deque(maxlen=1000)
     5                                               # losslog = open("losslog.txt","a")
     6                                               
     7      2720 5406845735.0    2e+06      2.8      for f,x in enumerate(coord_dataset):
     8      2720 7218270544.0    3e+06      3.8          fx = fwd(x)
     9      5440  109419631.0  20113.9      0.1          with tf.GradientTape() as tape:
    10      2720        4e+10    2e+07     23.1              y = leo(fx,training=True)
    11      2720        1e+10    4e+06      6.1              loss = eudist(fwd(y),fx)
    12      2720        8e+10    3e+07     42.2          gradient

In [273]:
# batch_size=1
# leo.load_weights("leo_v1-2-3/step_40000.keras")
# # # r = tf.cast(np.random.uniform(size=(batch_size,5)), "float32")

# inp = tf.constant([[4.,1.,0.]],dtype="float32")
# a = leo(inp)
# print(fwd(a))




In [274]:
# leo.save("leo_conclusory.keras")